#Project 2

Avik Bhattacharya

Updated on 2/21/2023 on the instruction of Professor Vasiliu.

##Imports

In [1]:
#Graphical libraries
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120
from IPython.display import Image
from IPython.display import display
plt.style.use('seaborn-white')

In [2]:
!pip install --upgrade --q scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 46.5 MB/s eta 0:00:00


In [3]:
#Other Used Libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.decomposition import PCA
from scipy.spatial import Delaunay
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import scipy.stats as stats 
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from scipy.interpolate import interp1d, griddata, LinearNDInterpolator, NearestNDInterpolator
from math import ceil
from scipy import linalg
from scipy.spatial import Delaunay

#Used in scikit compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [4]:
#May be used later.
lm = LinearRegression()
scale = StandardScaler()
qscale = QuantileTransformer()

In [5]:
#Datasets used
cars = pd.read_csv('drive/MyDrive/ADV Applied Machine Learning/Module 1/Day 3/cars.csv')
concrete = pd.read_csv('drive/MyDrive/ADV Applied Machine Learning/Module 1/Day 3/concrete.csv')

##Question 1

Adapt and modify the code for Gramfort’s version of Lowess to accommodate train and test sets with multidimensional features.

In [6]:
#Euclidean distance between all the observations in u and v.
def dist(u,v):
  if len(v.shape)==1: #If v is one dimensional, it is forced into a row vector.
    v = v.reshape(1,-1)
  d = np.array([np.sqrt(np.sum((u-v[i])**2,axis=1)) for i in range(len(v))])
  return d

Using the code we went over in class. I changed the weight kernel however so you could pick what kernel you want.

In [7]:
def lw_ag_md(x, y, xnew,f=2/3,iter=3, intercept=True, kernel='Tricubic'):

  n = len(x) #Number of observations
  r = int(ceil(f * n)) #Calculating the size of the neighborhood
  yest = np.zeros(n)

  if len(y.shape)==1: #Here we make column vector
    y = y.reshape(-1,1)

  if len(x.shape)==1: #Also making column vectors.
    x = x.reshape(-1,1)
  
  if intercept:
    x1 = np.column_stack([np.ones((len(x),1)),x])
  else:
    x1 = x

  #Finds the difference between each value of x, sorts them, and only gives you the values in the neighberhood.
  h = [np.sort(np.sqrt(np.sum((x-x[i])**2,axis=1)))[r] for i in range(n)]

  #Calculates the Euclidean distance and makes sure the weights don't go above 1 or under 0.
  w = np.clip(dist(x,x) / h, 0.0, 1.0)

  #Creates the weight using a kernel of your choice. Default is tricubic.
  if(kernel=='Tricubic'):
    w = (1 - w ** 3) ** 3
  elif(kernel=='Exponential'):
    w = np.exp(-(((w)**2)/2))
  elif(kernel=='Epanechnikov'):
    w = (3/4)*(1 - w ** 2)
  elif(kernel=='Quartic'):
    w = (15/16)*((1 - w ** 2) **2)
  elif(kernel=='Gaussian'):
    w = (1/(np.sqrt(2*np.pi)))*np.exp((-1/2)*(x**2))
  else:
    w = (1 - w ** 3) ** 3

  #Looping through all X-points
  delta = np.ones(n)
  for iteration in range(iter):
    for i in range(n):
      W = np.diag(w[:,i]).dot(np.diag(w[i,:]))#Updated
      b = np.transpose(x1).dot(W).dot(y)
      A = np.transpose(x1).dot(W).dot(x1)
      ##
      A = A + 0.0001*np.eye(x1.shape[1])  #If we want L2 regularization
      beta = linalg.solve(A, b)
      #beta, res, rnk, s = linalg.lstsq(A, b)
      yest[i] = np.dot(x1[i],beta)

    residuals = y - yest
    s = np.median(np.abs(residuals))
    delta = np.clip(residuals / (6.0 * s), -1, 1) #Clips the very high and low outliers.
    delta = (1 - delta ** 2) ** 2 #Very low residuals get centered at a weight of 1.

  if x.shape[1]==1:
    f = interp1d(x.flatten(),yest,fill_value='extrapolate')
    output = f(xnew)
  else:
    output = np.zeros(len(xnew))
    for i in range(len(xnew)):
      ind = np.argsort(np.sqrt(np.sum((x-xnew[i])**2,axis=1)))[:r]
      #Has Delauney triangulation work
      #Also prevents code from running too long.
      pca = PCA(n_components=3)
      x_pca = pca.fit_transform(x[ind])
      tri = Delaunay(x_pca,qhull_options='QJ')
      f = LinearNDInterpolator(tri,y[ind])
      output[i] = f(pca.transform(xnew[i].reshape(1,-1))) # the output may have NaN's where the data points from xnew are outside the convex hull of X
  if sum(np.isnan(output))>0:
    g = NearestNDInterpolator(x,y.ravel()) 
    # output[np.isnan(output)] = g(X[np.isnan(output)])
    output[np.isnan(output)] = g(xnew[np.isnan(output)])
  return output

In [8]:
x = cars.loc[:,'CYL':'WGT'].values
y = cars['MPG'].values

As shown below, the function works with test train splits.

In [9]:
xtrain, xtest, ytrain, ytest = tts(x,y,test_size=0.3,shuffle=True,random_state=123)
yhat = lw_ag_md(xtrain,ytrain,xtest,f=1/3,iter=3,intercept=True)

In [10]:
mse(ytest,yhat)

22.69720827114854

##Making Function SciKitLearn-compliant

I'm making the function SciKitLearn-compliant just like in class for the next question and the bonus question.

In [11]:
#Just like class but with a kernel paramater added.
class Lowess_AG_MD:
    def __init__(self, f = 1/10, iter = 3, intercept=True, kernel='Tricubic'):
        self.f = f
        self.iter = iter
        self.intercept = intercept
        self.kernel= kernel
    
    def fit(self, x, y):
        f = self.f
        iter = self.iter
        kernel = self.kernel
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        f = self.f
        iter = self.iter
        intercept = self.intercept
        kernel = self.kernel
        return lw_ag_md(x, y, x_new, f, iter, intercept, kernel)

    def get_params(self, deep=True):
    # suppose this estimator has parameters "f", "iter" , "intercept, and "kernel"
        return {"f": self.f, "iter": self.iter,"intercept":self.intercept,"kernel":self.kernel}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

##Question 2

Test your new function from 1) on some real data sets with k-Fold cross-validations.

I will be testing the function on both the cars and concrete datasets.

In [12]:
#Cars dataset
x = cars.loc[:,'CYL':'WGT'].values
y = cars['MPG'].values

In [13]:
#Testing function with k-Fold cross-validation on Cars dataset.
mse_lwr_cars = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
model_lw = Lowess_AG_MD(f=1/3,iter=1,intercept=True)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_lw.fit(xtrain,ytrain)
  yhat_lw = model_lw.predict(xtest)

  mse_lwr_cars.append(mse(ytest,yhat_lw))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression on the Cars dataset is : '+str(np.mean(mse_lwr_cars)))

The Cross-validated Mean Squared Error for Locally Weighted Regression on the Cars dataset is : 23.53685564215548


In [14]:
#Concrete dataset
x = concrete.loc[:,'cement':'age'].values
y = concrete['strength'].values

In [15]:
#Testing function with k-Fold cross-validation on Concrete dataset.
mse_lwr = []
kf = KFold(n_splits=5,shuffle=True,random_state=1234)
model_lw = Lowess_AG_MD(f=1/3,iter=1,intercept=True)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_lw.fit(xtrain,ytrain)
  yhat_lw = model_lw.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression on the Concrete dataset is : '+str(np.mean(mse_lwr)))

The Cross-validated Mean Squared Error for Locally Weighted Regression on the Concrete dataset is : 80.12810116114272


##Bonus Question

Create a SciKitLearn-compliant version of the function you wrote for 1) and test it with GridSearchCV from SciKitLearn.

I did the SciKitLearn compliance just before question 2.


In [16]:
#Testing GridSearch using Cars dataset
x = cars.loc[:,'CYL':'WGT'].values
y = cars['MPG'].values

In [17]:
#Creating the pipeline.
lwr_pipe = Pipeline([('zscores', StandardScaler()), ('lwr', Lowess_AG_MD())])

In [18]:
params = [{'lwr__f': [1/i for i in range(3,15)], 'lwr__iter': [1,2,3,4]}]

In [19]:
gs_lowess = GridSearchCV(lwr_pipe, param_grid=params, scoring='neg_mean_squared_error', cv=5)
gs_lowess.fit(x, y)
gs_lowess.best_params_

{'lwr__f': 0.3333333333333333, 'lwr__iter': 1}